In [2]:
import pandas as pd

# Step 1: Load expression matrix
expression_df = pd.read_csv("expression", sep="\t", index_col=0)

# Step 2: Transpose so rows = samples, columns = genes
expression_df = expression_df.transpose()

# Step 3: Clean sample IDs to match the format used in the response dataframe
expression_df.index = expression_df.index.str.replace("-01", "", regex=False)

# Step 4: Load response labels again (from earlier phenotype processing)
phenotype_df = pd.read_csv("phenotype", sep="\t")
response_df = phenotype_df[["sampleID", "primary_therapy_outcome_success"]].copy()
response_df.columns = ["sample", "outcome"]
response_df["sample"] = response_df["sample"].str.replace("-01", "", regex=False)

response_map = {
    "Complete Remission/Response": 1,
    "Partial Remission/Response": 1,
    "Progressive Disease": 0,
    "Stable Disease": 0
}
response_df["response"] = response_df["outcome"].map(response_map)
response_df = response_df.dropna(subset=["response"])

# Step 5: Merge expression matrix with response labels
expression_with_response = expression_df.merge(response_df[["sample", "response"]],
                                                left_index=True, right_on="sample")

# Step 6: Separate features and labels
X_expr = expression_with_response.drop(columns=["sample", "response"])
y_expr = expression_with_response["response"]

# Sanity check
print("Expression matrix shape (samples x genes):", X_expr.shape)
print("Label distribution:")
print(y_expr.value_counts())

Expression matrix shape (samples x genes): (483, 20530)
Label distribution:
response
1.0    425
0.0     58
Name: count, dtype: int64


T TEST

In [ ]:
from scipy.stats import ttest_ind

# Step 1: Split expression matrix by response group
responder_expr = X_expr[y_expr == 1]
nonresponder_expr = X_expr[y_expr == 0]

# Step 2: Perform t-tests across all genes
p_values = {}
for gene in X_expr.columns:
    t_stat, p_val = ttest_ind(responder_expr[gene], nonresponder_expr[gene], equal_var=False)
    p_values[gene] = p_val

# Step 3: Convert to DataFrame and filter
ttest_results = pd.DataFrame.from_dict(p_values, orient='index', columns=["p_value"])
ttest_results.sort_values("p_value", inplace=True)

# Optional: apply p-value cutoff
significant_genes_expr = ttest_results[ttest_results["p_value"] < 0.05].index.tolist()

# Step 4: Subset the expression matrix to only those genes
X_sig_expr = X_expr[significant_genes_expr]

# Summary
print("Total genes tested:", len(X_expr.columns))
print("Significant genes (p < 0.05):", len(significant_genes_expr))
print("Shape of filtered expression matrix:", X_sig_expr.shape)

Total genes tested: 20530
Significant genes (p < 0.05): 5072
Shape of filtered expression matrix: (483, 5072)


MODEL TRAINING ON FILTERED EXPRESSION DATA

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Step 1: Prepare data
X = X_sig_expr.values
y = y_expr.values

# Step 2: Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Step 3: Scale expression values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Step 5: Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Step 6: Evaluate
y_pred_prob = model.predict(X_test_scaled).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred_prob)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nTest AUC: {auc:.3f}")
print("\nConfusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


Epoch 1/50


/opt/anaconda3/envs/ml-env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7419 - auc: 0.6682 - loss: 0.5676 - val_accuracy: 0.8590 - val_auc: 0.7463 - val_loss: 0.6491
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8873 - auc: 0.7060 - loss: 0.6147 - val_accuracy: 0.8590 - val_auc: 0.7585 - val_loss: 0.6743
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9058 - auc: 0.8095 - loss: 0.3835 - val_accuracy: 0.8846 - val_auc: 0.7809 - val_loss: 0.7396
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9271 - auc: 0.9297 - loss: 0.2534 - val_accuracy: 0.8718 - val_auc: 0.8053 - val_loss: 0.5064
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9279 - auc: 0.9503 - loss: 0.1822 - val_accuracy: 0.8590 - val_auc: 0.8128 - val_loss: 0.5684
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9157 - auc: 0.9199 - loss: 0.2423 - val_accuracy: 0.8718 - val_auc: 0.7171 - val_loss: 0.8470
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 

CUT DOWN SIGNIFICANT GENES FROM 5000 TO MUCH LOWER

In [9]:
top_n = 500  # Or try 1000, 2000
top_genes_by_p = ttest_results.sort_values("p_value").head(top_n).index.tolist()
X_sig_expr_topN = X_expr[top_genes_by_p]
print("Top N significant genes by p-value:", len(X_sig_expr_topN.columns))

Top N significant genes by p-value: 500


RETRAIN ON TOP 500 GENES

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

# Step 1: Define features and labels
X = X_sig_expr_topN.values
y = y_expr.values

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Step 3: Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Compute class weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights_array))

# Step 5: Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Step 6: Train the model
history = model.fit(
    X_train_scaled, y_train,
    epochs=50, batch_size=16,
    validation_split=0.2,
    class_weight=class_weights,
    verbose=1
)

# Step 7: Evaluate
y_pred_prob = model.predict(X_test_scaled).flatten()
y_pred = (y_pred_prob >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred_prob)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nTest AUC: {auc:.3f}")
print("\nConfusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


Epoch 1/50


/opt/anaconda3/envs/ml-env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5813 - auc: 0.6501 - loss: 2.2358 - val_accuracy: 0.7564 - val_auc: 0.8745 - val_loss: 1.9329
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7225 - auc: 0.8095 - loss: 2.0144 - val_accuracy: 0.7564 - val_auc: 0.8745 - val_loss: 1.8069
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7424 - auc: 0.8383 - loss: 1.8254 - val_accuracy: 0.8333 - val_auc: 0.8772 - val_loss: 1.6819
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8180 - auc: 0.9036 - loss: 1.7328 - val_accuracy: 0.8205 - val_auc: 0.8908 - val_loss: 1.6094
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8160 - auc: 0.8673 - loss: 1.6086 - val_accuracy: 0.8333 - val_auc: 0.9138 - val_loss: 1.5241
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8470 - auc: 0.9501 - loss: 1.5152 - val_accuracy: 0.8077 - val_auc: 0.9152 - val_loss: 1.4827
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 